<a href="https://colab.research.google.com/github/PedroTechy/CarrisInsight/blob/streaming_development/spark_jobs/extract_carris_vehicles.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Step 1: Authenticate with Google Cloud


In [8]:
!gcloud auth application-default login

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=764086051850-6qr4p6gpi6hn506pt8ejuq83di341hur.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8085%2F&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login&state=J6N3ibxNqLovcG4msfcTEHEetXC11r&access_type=offline&code_challenge=ycYyvB3j8Kzo6bTw0QilWjlp2t0cd9QkW3mQQutWDqE&code_challenge_method=S256


Credentials saved to file: [/Users/ineslopes/.config/gcloud/application_default_credentials.json]

These credentials will be used by any library that requests Application Default Credentials (ADC).
Cannot add the project "data-eng-dev-437916" to ADC as the quota project because the account in ADC does not have the "serviceusage.services.use" permission on this project. You might receive a "quota_exceeded" or "API not enabled" error

# Step 2: Install Spark

In [9]:
!pip install pyspark

# Step 3: Setup Spark Env and Imports

In [1]:
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType, LongType
from pyspark.sql.functions import min, max, first, last, col, window, from_unixtime, to_timestamp, count, udf
import os

In [2]:
home_directory = os.getenv("HOME")

spark = SparkSession.builder \
    .appName('pyspark-run-with-gcp-bucket') \
    .config("spark.jars", "https://storage.googleapis.com/hadoop-lib/gcs/gcs-connector-hadoop3-latest.jar") \
    .config("spark.sql.repl.eagerEval.enabled", True) \
    .getOrCreate()

gs_input_path = "gs://edit-de-project-streaming-data/carris-vehicles"
spark._jsc.hadoopConfiguration().set("google.cloud.auth.service.account.json.keyfile",
                                     f"{home_directory}/.config/gcloud/application_default_credentials.json")

25/01/21 18:57:24 WARN Utils: Your hostname, Iness-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.1.9 instead (on interface en0)
25/01/21 18:57:24 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/01/21 18:57:30 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/01/21 18:57:32 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
25/01/21 18:57:32 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [7]:
#Cleaning the path to ensure clean directories
!rm -rf content/lake/

# Step 4: Define Schema and User Defined Functions

In [109]:
# Define the schema for your JSON files

schema = StructType([
    StructField("bearing", FloatType(), True),
    StructField("block_id", StringType(), True),
    StructField("current_status", StringType(), True),
    StructField("id", StringType(), True),
    StructField("lat", FloatType(), True),
    StructField("line_id", StringType(), True),
    StructField("lon", FloatType(), True),
    StructField("pattern_id", StringType(), True),
    StructField("route_id", StringType(), True),
    StructField("schedule_relationship", StringType(), True),
    StructField("shift_id", StringType(), True),
    StructField("speed", FloatType(), True),
    StructField("stop_id", StringType(), True),
    StructField("timestamp", LongType(), True),
    StructField("trip_id", StringType(), True)
])

In [171]:
def get_stops(spark):
    url = f"https://api.carrismetropolitana.pt/stops"
    response = requests.get(url)

    filtered = [{'stop_id': stop['stop_id'], 'stop_lat': float(
        stop['lat']), 'stop_lon': float(stop['lon'])} for stop in json.loads(response.text)]
    
    schema = StructType([
    StructField("stop_id", StringType(), True),
    StructField("stop_lat", FloatType(), True),
    StructField("stop_lon", FloatType(), True)])

    stops = spark.createDataFrame(filtered, schema=schema)

    return stops
stops = get_stops(spark)


In [133]:
def aggregate_data(df):
    window_spec = window("timestamp", "2 minutes", "10 seconds")

    transformed = (df.withWatermark("timestamp", "3 minutes")
    .groupBy("id", "trip_id", window_spec)
    .agg(
        max("current_status").alias("current_status"),
        max("route_id").alias("route_id"),
        max("stop_id").alias("stop_id"),
        min("timestamp").alias("first_timestamp"),
        max("timestamp").alias("last_timestamp"),
        first("lat").alias("first_lat"),
        first("lon").alias("first_lon"),
        last("lat").alias("last_lat"),
        last("lon").alias("last_lon")
    )
    )
    return transformed

In [1]:
import numpy as np
np.radi

In [2]:
def calculate_distances_without_udf(df):
    print("started transform")
    dist_df = (df
               .withColumn(
                   "distance_km",
                   6371.0 * (2 * math.atan2(
                       math.sqrt(

                           np.sin(
                               (np.radians(col("latitude_2")) - np.radians(col("latitude_1"))) / 2)**2 +

                           np.cos(np.radians(col("latitude_1"))) * np.cos(np.radians(col("latitude_2"))) *
                           np.sin((np.radians(col("longitude_2")) -
                                     np.radians(col("longitude_1")))/2)**2

                       ),
                       math.sqrt(1 - (np.sin(
                           (np.radians(col("latitude_2")) - np.radians(col("latitude_1"))) / 2)**2 +

                           np.cos(np.radians(col("latitude_1"))) * np.cos(np.radians(col("latitude_2"))) *
                           np.sin((np.radians(col("longitude_2")) - np.radians(col("longitude_1")))/2)**2))))
               )
               .withColumn(
                   "time_delta", col("last_timestamp").cast("long") -
                   col("first_timestamp").cast("long")
               ).withColumn("average_speed", col("distance") / (col("time_delta") / 3600))
               )
    print("ended transform")

    return dist_df

In [3]:
def calculate_distances(df):
    print("started transform")
    dist_df = (df.withColumn(
        "distance",
        haversine_udf(col("first_lat"), col("first_lon"), col("last_lat"), col("last_lon")
        ))
    .withColumn(
        "time_delta", col("last_timestamp").cast("long") - col("first_timestamp").cast("long")
    ).withColumn("average_speed", col("distance") / (col("time_delta") / 3600))
        )
    print("ended transform")

    return dist_df



## Step 5: Start streaming and transformation process

In [ ]:
df = (spark.readStream.option("maxFilesPerTrigger", 1)
    .format("json")
    .schema(schema)
    .load(gs_input_path))
print("will start streaming")


transformed_df = df.withColumn("timestamp", to_timestamp(from_unixtime("timestamp")))
# Write a df with the datetype transform only
print("Transformed data")


# Group by vehicle ID and window, then get the first and last timestamps and lat/lon values
result_df = (
    transformed_df.transform(aggregate_data).transform(calculate_distances)

)

query = (result_df.writeStream
.outputMode('append')
.option('checkpointLocation', 'content/lake/processing/vehicles_checkpoint')
.trigger(processingTime='10 seconds')
.start('content/lake/processing/vehicles/data')
)

query.awaitTermination(30)

query.stop()


## Step 6: Read and verify results

In [ ]:
df = spark.read.format("parquet").load("content/lake/processing/vehicles/data")


In [ ]:
df.count()

In [ ]:
view = (df
 .orderBy("last_timestamp", ascending=False)
 .dropDuplicates(["id"])
 .join(stops, how='left', on='stop_id')
 .withColumn("distance_till_stop", haversine_udf(col("last_lat"), col('last_lon'), col('stop_lat'), col('stop_lon')))
 .withColumn("stop_eta", col("distance_till_stop") / col("average_speed") *60)
 )


In [ ]:
view.show()